In [59]:
import os
import warnings
from pathlib import Path
from collections import defaultdict

import numpy as np
import pandas as pd

import datamol as dm
from catboost import Pool, CatBoostRegressor, cv

import autosklearn.classification
import autosklearn.regression

from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_absolute_error, roc_auc_score
from rdkit.Chem import SaltRemover

from molfeat.store.modelstore import ModelStore
from molfeat.calc import FPCalculator
from molfeat.trans import MoleculeTransformer, FPVecTransformer
from molfeat.trans.pretrained.hf_transformers import PretrainedHFTransformer, HFModel

In [2]:
warnings.simplefilter("ignore")
os.environ["PYTHONWARNINGS"] = "ignore"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
dm.disable_rdkit_log()

In [3]:
# Set path to this notebook
HERE = Path(_dh[-1])
DATA = HERE / "data"

In [4]:
def load_dataset(uri: str, readout_col: str):
    """ Loads the MoleculeNet dataset """
    df = pd.read_csv(uri)
    smiles = df["smiles"].values
    y = df[readout_col].values
    return smiles, y


def preprocess_smiles(smi: str) -> str:
    """ Preprocesses the SMILES string """
    mol = dm.to_mol(smi, ordered=True, sanitize=False)    
    try: 
        mol = dm.sanitize_mol(mol)
    except:
        mol = None
            
    if mol is None: 
        return
        
    mol = dm.standardize_mol(mol, disconnect_metals=True)
    remover = SaltRemover.SaltRemover()
    mol = remover.StripMol(mol, dontRemoveEverything=True)

    return dm.to_smiles(mol)


def scaffold_split(smiles):
    """In line with common practice, we will use the scaffold split to evaluate our models"""
    scaffolds = [dm.to_smiles(dm.to_scaffold_murcko(dm.to_mol(smi))) for smi in smiles]
    splitter = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
    return next(splitter.split(smiles, groups=scaffolds))

In [5]:
smiles, y = load_dataset(DATA / "EGFR_compounds_lipinski.csv", "pIC50")

In [6]:
smiles = np.array([preprocess_smiles(smi) for smi in smiles])
smiles = np.array([smi for smi in smiles if dm.to_mol(smi) is not None])

In [7]:
calc = FPCalculator("ecfp")
ecfp_transf = MoleculeTransformer(calc, n_jobs=-1)

In [9]:
train_ind, test_ind = scaffold_split(smiles)

In [10]:
feats_ecfp, ind_ecfp = ecfp_transf(smiles, ignore_errors=True)
X = np.array(feats_ecfp)[ind_ecfp]
X_train = X[train_ind]
X_test = X[test_ind]
y_train = y[train_ind]
y_test = y[test_ind]

In [ ]:
# Train
automl = autosklearn.regression.AutoSklearnRegressor(
    memory_limit=24576, 
    # For practicality’s sake, limit this to 5 minutes! 
    # (x3 = 15 min in total)
    time_left_for_this_task=180,  
    n_jobs=1,
    seed=1,
)
automl.fit(X_train, y_train)

# Predict and evaluate
y_hat = automl.predict(X_test)

In [ ]:
# Evaluate
mae = mean_absolute_error(y_test, y_hat)
mae

## Combine with Catboost

In [11]:
X_train.shape, X_test.shape

((3707, 2048), (928, 2048))

In [14]:
# initialize Pool
cat_features = list(range(2048))
train_pool = Pool(X_train, y_train, cat_features=cat_features)
test_pool = Pool(X_test, y_test, cat_features=cat_features)

In [16]:
# specify the training parameters 
model = CatBoostRegressor(iterations=10000,
                          depth=3,
                          loss_function='MAE',
                          verbose=False)
#train the model
model.fit(train_pool, 
          eval_set=test_pool,
          plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [46]:
model.get_best_score()

{'learn': {'MAE': 0.3980797927793916},
 'validation': {'MAE': 0.7235377879657542}}

### Catboost with cross validation

In [ ]:
cv_dataset = Pool(data=X, label=y, cat_features=cat_features)

params = {
    "iterations": 10000,
    "depth": 3,
    "loss_function": "MAE",
    "verbose": False
}

scores = cv(cv_dataset,
            params,
            fold_count=5, 
            plot="True")

In [ ]:
scores

## Try different calculators

In [57]:
store = ModelStore("/home/adrien/molfeat-store-prod/artifacts/")

[]

In [ ]:
modelcard = {}
store.register(modelcard=modelcard, model="")

In [60]:
chemBERTaModel = HFModel(name='ChemBERTa-77M-MLM', store=store)

In [64]:
all_trans = {
    "ecfp:6": FPVecTransformer(kind="ecfp:6", n_jobs=-1),
    "mordred": FPVecTransformer(kind="mordred", replace_nan=True, n_jobs=-1),
    #"ChemBERTa-77M-MLM": PretrainedHFTransformer(kind=chemBERTaModel, notation='smiles')
}

In [67]:
all_trans

{'ecfp:6': FPVecTransformer(kind="ecfp:6", length=2000, dtype=np.float32),
 'mordred': FPVecTransformer(kind="mordred", length=2000, dtype=np.float32)}

In [72]:
def transform_and_fit(trans, smiles, train_ind, test_ind):
    # Transform features
    features, indices = trans(smiles, ignore_errors=True)
    X = np.array(features)[indices]

    # Split dataset
    X_train = X[train_ind]
    X_test = X[test_ind]
    y_train = y[train_ind]
    y_test = y[test_ind]

    # Initialize Pools
    cat_features = []
    #cat_features = list(range(X_train.shape[1]))
    train_pool = Pool(X_train, y_train, cat_features=cat_features)
    test_pool = Pool(X_test, y_test, cat_features=cat_features)

    # Specify the training parameters 
    model = CatBoostRegressor(iterations=3000,
                              depth=3,
                              loss_function='MAE',
                              verbose=True)
    # Train the model
    model.fit(train_pool, 
              eval_set=test_pool,
              plot=False)
    
    # Extract best score
    return model.get_best_score()

In [68]:
scores = defaultdict(list)

for name, trans in all_trans.items():
    best_score = transform_and_fit(trans, smiles, train_ind, test_ind)
    scores["name"].append(name)
    scores["train-MAE"].append(best_score["learn"]["MAE"])
    scores["test-MAE"].append(best_score["validation"]["MAE"])

0:	learn: 1.1767248	test: 1.1803397	best: 1.1803397 (0)	total: 9.81ms	remaining: 29.4s
1:	learn: 1.1629794	test: 1.1678641	best: 1.1678641 (1)	total: 17.8ms	remaining: 26.6s
2:	learn: 1.1501758	test: 1.1560504	best: 1.1560504 (2)	total: 24.4ms	remaining: 24.4s
3:	learn: 1.1373711	test: 1.1446193	best: 1.1446193 (3)	total: 30.3ms	remaining: 22.7s
4:	learn: 1.1261393	test: 1.1353410	best: 1.1353410 (4)	total: 35.9ms	remaining: 21.5s
5:	learn: 1.1148420	test: 1.1252512	best: 1.1252512 (5)	total: 40.9ms	remaining: 20.4s
6:	learn: 1.1052631	test: 1.1159928	best: 1.1159928 (6)	total: 46.6ms	remaining: 19.9s
7:	learn: 1.0953271	test: 1.1072934	best: 1.1072934 (7)	total: 51.3ms	remaining: 19.2s
8:	learn: 1.0867565	test: 1.0986320	best: 1.0986320 (8)	total: 56.2ms	remaining: 18.7s
9:	learn: 1.0776016	test: 1.0903319	best: 1.0903319 (9)	total: 63.5ms	remaining: 19s
10:	learn: 1.0696956	test: 1.0823201	best: 1.0823201 (10)	total: 67.6ms	remaining: 18.4s
11:	learn: 1.0617562	test: 1.0745214	best: 

In [69]:
scores

defaultdict(list,
            {'name': ['ecfp:6', 'mordred'],
             'train-MAE': [0.48742393356403346, 0.46736693700505044],
             'test-MAE': [0.769891690390054, 0.8017289970268583]})

In [86]:
desc2D_transf = FPVecTransformer(kind='desc2D', dtype=float)

In [88]:
transform_and_fit(desc2D_transf, smiles, train_ind, test_ind)

0:	learn: 1.1798443	test: 1.1827495	best: 1.1827495 (0)	total: 4ms	remaining: 12s
1:	learn: 1.1700294	test: 1.1732639	best: 1.1732639 (1)	total: 7.56ms	remaining: 11.3s
2:	learn: 1.1598662	test: 1.1622405	best: 1.1622405 (2)	total: 10.7ms	remaining: 10.7s
3:	learn: 1.1480755	test: 1.1503837	best: 1.1503837 (3)	total: 13.9ms	remaining: 10.4s
4:	learn: 1.1385896	test: 1.1391276	best: 1.1391276 (4)	total: 17.1ms	remaining: 10.2s
5:	learn: 1.1291221	test: 1.1293273	best: 1.1293273 (5)	total: 20.3ms	remaining: 10.1s
6:	learn: 1.1187881	test: 1.1205522	best: 1.1205522 (6)	total: 23.4ms	remaining: 10s
7:	learn: 1.1093498	test: 1.1127889	best: 1.1127889 (7)	total: 26.2ms	remaining: 9.8s
8:	learn: 1.1017946	test: 1.1049934	best: 1.1049934 (8)	total: 29.1ms	remaining: 9.68s
9:	learn: 1.0939220	test: 1.0981214	best: 1.0981214 (9)	total: 32ms	remaining: 9.56s
10:	learn: 1.0860407	test: 1.0912544	best: 1.0912544 (10)	total: 34.8ms	remaining: 9.46s
11:	learn: 1.0781745	test: 1.0846830	best: 1.084683

{'learn': {'MAE': 0.5137786515012782},
 'validation': {'MAE': 0.7970426797086888}}

In [90]:
scaffoldkeys_trans = MoleculeTransformer(featurizer='scaffoldkeys', dtype=float)
transform_and_fit(desc2D_transf, smiles, train_ind, test_ind)

0:	learn: 1.1798443	test: 1.1827495	best: 1.1827495 (0)	total: 6.53ms	remaining: 19.6s
1:	learn: 1.1700294	test: 1.1732639	best: 1.1732639 (1)	total: 12.7ms	remaining: 19.1s
2:	learn: 1.1598662	test: 1.1622405	best: 1.1622405 (2)	total: 19.1ms	remaining: 19.1s
3:	learn: 1.1480755	test: 1.1503837	best: 1.1503837 (3)	total: 24.8ms	remaining: 18.6s
4:	learn: 1.1385896	test: 1.1391276	best: 1.1391276 (4)	total: 29.7ms	remaining: 17.8s
5:	learn: 1.1291221	test: 1.1293273	best: 1.1293273 (5)	total: 34.6ms	remaining: 17.3s
6:	learn: 1.1187881	test: 1.1205522	best: 1.1205522 (6)	total: 38.3ms	remaining: 16.4s
7:	learn: 1.1093498	test: 1.1127889	best: 1.1127889 (7)	total: 41.4ms	remaining: 15.5s
8:	learn: 1.1017946	test: 1.1049934	best: 1.1049934 (8)	total: 44.5ms	remaining: 14.8s
9:	learn: 1.0939220	test: 1.0981214	best: 1.0981214 (9)	total: 47.4ms	remaining: 14.2s
10:	learn: 1.0860407	test: 1.0912544	best: 1.0912544 (10)	total: 50.7ms	remaining: 13.8s
11:	learn: 1.0781745	test: 1.0846830	best

{'learn': {'MAE': 0.5137786515012782},
 'validation': {'MAE': 0.7970426797086888}}

In [91]:
maccs_transf = FPVecTransformer(kind='maccs', dtype=float)
transform_and_fit(desc2D_transf, smiles, train_ind, test_ind)

0:	learn: 1.1798443	test: 1.1827495	best: 1.1827495 (0)	total: 10.3ms	remaining: 30.8s
1:	learn: 1.1700294	test: 1.1732639	best: 1.1732639 (1)	total: 20.2ms	remaining: 30.3s
2:	learn: 1.1598662	test: 1.1622405	best: 1.1622405 (2)	total: 28.4ms	remaining: 28.3s
3:	learn: 1.1480755	test: 1.1503837	best: 1.1503837 (3)	total: 33.7ms	remaining: 25.2s
4:	learn: 1.1385896	test: 1.1391276	best: 1.1391276 (4)	total: 38.2ms	remaining: 22.9s
5:	learn: 1.1291221	test: 1.1293273	best: 1.1293273 (5)	total: 42.1ms	remaining: 21s
6:	learn: 1.1187881	test: 1.1205522	best: 1.1205522 (6)	total: 45.5ms	remaining: 19.5s
7:	learn: 1.1093498	test: 1.1127889	best: 1.1127889 (7)	total: 48.6ms	remaining: 18.2s
8:	learn: 1.1017946	test: 1.1049934	best: 1.1049934 (8)	total: 51.6ms	remaining: 17.2s
9:	learn: 1.0939220	test: 1.0981214	best: 1.0981214 (9)	total: 54.5ms	remaining: 16.3s
10:	learn: 1.0860407	test: 1.0912544	best: 1.0912544 (10)	total: 57.6ms	remaining: 15.6s
11:	learn: 1.0781745	test: 1.0846830	best: 

{'learn': {'MAE': 0.5137786515012782},
 'validation': {'MAE': 0.7970426797086888}}

In [ ]:
# Find a featurizer and learn how to use it
model_card = store.search(name="ChemBERTa-77M-MLM")[0]
print(model_card.usage())